### MGCeek
#### Take Specialized Metabolite gene pHMM hits (from (pl)anti-SMASH) and genomic gff to observe patterns in metabolic gene clustering   
#### Author=Bob_Auber
#### Date=07.08.2020

In [1]:
#this uses iPython magic to make plots appear inline
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import subprocess
import sys
import numpy as np
import matplotlib.patches as patches
import gzip
import fileinput
import glob
import os
from scipy import stats
import re
from collections import OrderedDict
import statistics
import seaborn as sns

def runCMD(cmd):
    val = subprocess.Popen(cmd, shell=True).wait()
    if val == 0:
        pass
    else:
        print ('command failed')
        print (cmd)
        sys.exit(1)


## Approach:
## Go through genome
### 1) Identitfy genes with SM HMM (SMG)
### 2) Create a gene window X genes/basepairs up and downstream of each SMG
### 3) Consolidate overlapping windows and observe instances of SMGs in windows 

In [2]:
Rootdir='/depot/jwisecav/data/rauber/projects/prymnesium/genomes/2797_phase/v0.5/MGC/'
gff='/depot/jwisecav/data/rauber/projects/prymnesium/genomes/2797_phase/v0.5/UTEX2797_genomic_v0.5.gff'
window_size=50000 # this will capture genes that are X genes/basepairs upstream AND downstream of each SMG
basepairs=True

## Define genes of interest to use as seeds for identifying gene windows of interest
#### These genes are stored into gene_hmm_dict
#### If using other gene sets, replace gene_hmm_dict with a dict of your choice in the format of 'geneID':[annotation1,annotation2,...]

In [3]:
## Create dictionaries containing info of genes with SM hmms (SMGS)
plant_out_files=glob.glob('/depot/jwisecav/data/rauber/projects/prymnesium/genomes/2797_phase/v0.5/annotation/hmms/plant/*hmm_out')
fungal_out_files=glob.glob('/depot/jwisecav/data/rauber/projects/prymnesium/genomes/2797_phase/v0.5/annotation/hmms/fungal/*hmm_out')
hmm_selection='all' #plant or fungi or KEGG or all

In [4]:
if hmm_selection == 'plant' or hmm_selection == 'all':
    plant_gene_hmm_dict={}
    for file in plant_out_files:
        with open(file) as sources:
            lines=sources.readlines()
            for line in lines:
                if line.startswith('UTEX') is True:
                    #print(line)
                    gene=line.split()[0].split('.')[0]
                    #print(gene)
                    evalue=float(line.split()[4])
                    #print(evalue)
                    HMM=line.split()[2]
                    #print(HMM)
                    
                    
                    if evalue < 1e-3:
                        if gene in plant_gene_hmm_dict:
                            plant_gene_hmm_dict[gene].append(HMM)
                        else:
                            plant_gene_hmm_dict[gene]=[HMM]
                        
if hmm_selection == 'fungi' or hmm_selection == 'all':                        
    fungal_gene_hmm_dict={}
    for file in fungal_out_files:
        with open(file) as sources:
            lines=sources.readlines()
            for line in lines:
                if line.startswith('UTEX') is True:
                    #print(line)
                    gene=line.split()[0].split('.')[0]
                    #print(gene)
                    evalue=float(line.split()[4])
                    #print(evalue)
                    HMM=line.split()[2]
                    #print(HMM)
                    
                    
                    if evalue < 1e-3:
                        if gene in fungal_gene_hmm_dict:
                            fungal_gene_hmm_dict[gene].append(HMM)
                            
                        else:
                            fungal_gene_hmm_dict[gene]=[HMM]


In [5]:
print(len(plant_gene_hmm_dict), 'genes have a plant SM pHMM')

1230 genes have a plant SM pHMM


In [6]:
print(len(fungal_gene_hmm_dict), 'genes have a fungal SM pHMM')

1707 genes have a fungal SM pHMM


In [7]:
# construct non-redundant HMM dict dependent on plant or fungal input
gene_hmm_dict={}
if hmm_selection == 'plant' or hmm_selection == 'all':
    #print('yay')
    for gene in plant_gene_hmm_dict:
        hmms=plant_gene_hmm_dict[gene]
        for hmm in set(hmms):
            if gene in gene_hmm_dict:
                gene_hmm_dict[gene].append(hmm)
            else:
                gene_hmm_dict[gene]=[hmm]

    
if hmm_selection == 'fungi' or hmm_selection == 'all':
    #print('yay')
    for gene in fungal_gene_hmm_dict:
        hmms=fungal_gene_hmm_dict[gene]
        for hmm in set(hmms):
            if gene in gene_hmm_dict:
                gene_hmm_dict[gene].append(hmm)
            else:
                gene_hmm_dict[gene]=[hmm]

In [10]:
print(len(gene_hmm_dict), 'genes have either a plant or fungal SM pHMM hit')

2400 genes have either a plant or fungal SM pHMM hit


## Parsing of GFF

In [13]:
#Generate gene:contig and gene:coordinate dictionaries 
gene_coord_dict={}
gene_contig_dict={}

count=0
with open(gff, 'r') as sources:
    lines=sources.readlines()
    for line in lines:
        if line.startswith('#') is False:
            contig=line.split('\t')[0]
            feature=line.split('\t')[2]
            start=float(line.split('\t')[3])
            
            if feature=='gene':
                #I am pulling from the ID=gene in last gff column (first comma delimited info)
                gene=line.split('\t')[-1].split(';')[0].split('=')[1]
                gene_coord_dict[gene]=start
                gene_contig_dict[gene]=contig
                count+=1
print(count, 'genes read from gff file')

45535 genes read from gff file


In [16]:
#Identify genes +-X upstream and downstream of each SM gene and put in SMG_gene_window_dict

SMG_gene_window_dict={}
SMG_contig_window_dict={}

#####EXTRACT COORDINATE INFO OF GENES######
for SMG in gene_hmm_dict:
    SMG_contig=gene_contig_dict[SMG]
    SMG_start=gene_coord_dict[SMG]
############################################
    
    ##########EXTRACT RELATIVE COORDINATES TO SMG##############
    coordinates=[]
    trimmed_coordinates=[]
    relative_coord_dict={}
    for gene in gene_contig_dict.keys():#for all genes
        contig=gene_contig_dict[gene]
        start=gene_coord_dict[gene]
        
        if contig == SMG_contig:
            relative_coord=-(SMG_start-start)
            
            if relative_coord in relative_coord_dict.keys():
                relative_coord_dict[relative_coord].append(gene)
            else:
                relative_coord_dict[relative_coord]=[gene]
            coordinates.append(relative_coord)
    coordinates.sort()       
    #print(len(coordinates), 'genes were found on the contig containing', SMG)
    ##########################################################
    
    
    
    ########FIND NUMBER OF GENES UPSTREAM AND DOWNSTREAM OF WINDOW############
    ahead=0
    behind=0
    for coordinate in coordinates:
        if basepairs is False:
            if coordinate < 0:
                behind+=1
            if coordinate > 0:
                ahead+=1

        if basepairs is True:
            if coordinate < -window_size:
                behind+=1
            if coordinate > window_size:
                ahead+=1
                
    #print(behind, 'genes were behind', SMG)
    #print(ahead, 'genes were ahead', SMG)
    #############################################WARNING MESSAGES
    #if basepairs is False:
    #    if ahead < window_size:
    #        print('WARNING! Size of gene window ahead', SMG, 'will be shortend to', ahead, 'genes instead of the specified window size:', window_size)
    #    if behind < window_size:
    #        print('WARNING! Size of gene window behind', SMG, 'will be shortend to', behind, 'genes instead of the specified window size:', window_size)
    #if basepairs is True:
    #    if max(coordinates) < window_size:
    #        print('WARNING! Size of window ahead', SMG, 'will be shortend to', abs(max(coordinates)), 'basepairs instead of the specified window size:', window_size)
    #    if min(coordinates) > -window_size:
    #        print('WARNING! Size of window behind', SMG, 'will be shortend to', abs(min(coordinates)), 'basepairs instead of the specified window size:', window_size)     
    ############################################
    
    
    
    
    ###########WINDOW TRIMMING###############
    if basepairs is False:
        if ahead > window_size:
            extra=ahead-window_size
            #print(extra, 'extra genes ahead')
            coordinates=coordinates[:-extra]
    
        if behind > window_size:
            extra=behind-window_size
            #print(extra, 'extra genes behind')
            coordinates=coordinates[extra:]
            
    if basepairs is True:
        if ahead > 0:
            #print(extra, 'extra genes ahead')
            coordinates=coordinates[:-ahead]
        if behind > 0:
            coordinates=coordinates[behind:]
    #print(len(coordinates))
    #######################################

    
    
    ##########GENE WINDOW ASSOCIATIONS##########
    for coordinate in coordinates:
        genes=relative_coord_dict[coordinate]
        for gene in genes:    
            if SMG in SMG_gene_window_dict:
                SMG_gene_window_dict[SMG].append(gene)
            else:
                SMG_gene_window_dict[SMG]=[gene]
        ###########################################
        
     

    

#### This dict contains each SMG and the genes up/downstream according to thresholds

In [31]:
#SMG_gene_window_dict

## Consolidate overlapping windows into one. (SMGs must be in same window to be combined)

In [18]:
count=0
raw_consolidated_SMG_gene_window_dict={}
for SMG in SMG_gene_window_dict:
    count+=1

    raw_consolidated_SMG_gene_window_dict['window_'+str(count)]=set()
    raw_consolidated_SMG_gene_window_dict['window_'+str(count)].add(SMG)
    window_genes=SMG_gene_window_dict[SMG]
    
    #find all SMGS in window of SMG
    for gene in window_genes:
        if gene == SMG:
            continue
        else:
            if gene in SMG_gene_window_dict.keys():
                raw_consolidated_SMG_gene_window_dict['window_'+str(count)].add(gene)
    
   
    
    i=1
    while i == 1: #now continually iterate through all SMGs in window to see if they have neighboring SMGs too
        
        
        to_add_genes=set()
        for window in raw_consolidated_SMG_gene_window_dict['window_'+str(count)]:
            window_genes=SMG_gene_window_dict[window]
            for gene in window_genes:
                if gene in raw_consolidated_SMG_gene_window_dict['window_'+str(count)]: #if already identified in same window
                    continue
                if gene == SMG:
                    continue
                else:
                    if gene in SMG_gene_window_dict.keys():
                        to_add_genes.add(gene)
                        
        #now add new SMGs found next to neighboring SMGs
        if len(to_add_genes) == 0:
            i = 0
            #print('done')
        else:
            #print(len(to_add_genes))
            i=1 #iterate again
            for gene in to_add_genes:
                raw_consolidated_SMG_gene_window_dict['window_'+str(count)].add(gene)
    
                    
    
        
        

    

In [19]:
len(raw_consolidated_SMG_gene_window_dict)

2400

### Though windows are consolidated, they will be redudant for each overlapping SMG 
### Lets identitify them and only retain unique windows

In [23]:
overlap_sets=[]
for window in raw_consolidated_SMG_gene_window_dict:
    overlap_set=raw_consolidated_SMG_gene_window_dict[window]
    if overlap_set in overlap_sets:
        continue
    else:
        overlap_sets.append(overlap_set)
print(len(overlap_sets))

1281


In [24]:
overlap_genes=[]
count=0
for overlap_set in overlap_sets:
    for gene in overlap_set:
        count+=1
        overlap_genes.append(gene)
print(len(overlap_genes))
print(len(set(overlap_genes)))#sanity check

2400
2400


In [25]:
consolidated_SMG_gene_window_dict={}
count=0
for overlap_set in overlap_sets:
    count+=1
    consolidated_SMG_gene_window_dict['window_'+str(count)]=set()
    for SMG in overlap_set:
        window_genes=SMG_gene_window_dict[SMG]
        for gene in window_genes:
            consolidated_SMG_gene_window_dict['window_'+str(count)].add(gene)


In [26]:
print(len(consolidated_SMG_gene_window_dict), 'unique SMG windows found')

1281 unique SMG windows found


In [28]:
#consolidated_SMG_gene_window_dict

In [29]:
#See how many SMGs are in each consolidated window
SMG_consolidated_window_counts=[]
SMG_consolidated_window_plot_dict={}

for window in consolidated_SMG_gene_window_dict:

    window_genes=consolidated_SMG_gene_window_dict[window]
    count=0
    for gene in window_genes:
        if gene in gene_hmm_dict:
            
            count+=1
    SMG_consolidated_window_counts.append(count)


for threshold in range(0,max(SMG_consolidated_window_counts)+1):
    count=0
    for num in SMG_consolidated_window_counts:
        if num == 0:
            num += 1 
        if num == threshold:
            count+=1
    if count != 0:
        print(count, '(', (count/len(SMG_consolidated_window_counts))*100, '%) consolidated windows contain' , threshold ,'SMGs')
        SMG_consolidated_window_plot_dict[str(threshold)]=count
        

691 ( 53.942232630757225 %) consolidated windows contain 1 SMGs
314 ( 24.512099921935988 %) consolidated windows contain 2 SMGs
146 ( 11.397345823575332 %) consolidated windows contain 3 SMGs
74 ( 5.776736924277908 %) consolidated windows contain 4 SMGs
27 ( 2.107728337236534 %) consolidated windows contain 5 SMGs
15 ( 1.1709601873536302 %) consolidated windows contain 6 SMGs
4 ( 0.312256049960968 %) consolidated windows contain 7 SMGs
5 ( 0.39032006245121 %) consolidated windows contain 8 SMGs
2 ( 0.156128024980484 %) consolidated windows contain 9 SMGs
2 ( 0.156128024980484 %) consolidated windows contain 11 SMGs
1 ( 0.078064012490242 %) consolidated windows contain 14 SMGs


### Example how to write output to files

In [ ]:
#with open(Rootdir + '10_gene_all_hmm_and_KEGG_SMG_consolidated_windows.txt', 'w') as f:
#    for window in consolidated_SMG_gene_window_dict:
#        window_genes=consolidated_SMG_gene_window_dict[window]
#        f.write(window+':'+','.join(window_genes)+'\n')
        
#with open(Rootdir + '50kb_all_hmm_SMG_consolidated_windows.txt', 'w') as f:
#    for window in consolidated_SMG_gene_window_dict:
#        window_genes=consolidated_SMG_gene_window_dict[window]
#        f.write(window+':'+','.join(window_genes)+'\n')